In [306]:
from bs4 import BeautifulSoup as bs
from urllib.request import Request, urlopen
import requests

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver import Keys, ActionChains
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
 
import re
from random import randint
from time import sleep
import json
import pandas as pd

In [279]:
#Search details
location = "gdansk"
searching_object = "mieszkanie"
offer_type = "wynajem"

#Additional info to search pages
offers_limit = 24
pageNo = 1
pageNo_End = 2

#URL with the list of offers
main_url = f"https://www.otodom.pl/pl/oferty/{offer_type}/{searching_object}/{location}?distanceRadius=0&page={pageNo}&limit={offers_limit}&market=ALL&ownerTypeSingleSelect=ALL&locations=%5Bcities_6-40%5D&by=DEFAULT&direction=DESC&viewType=listing"

In [281]:
path = Service(executable_path="D:\ChromeDriver\chromedriver.exe")
driver = webdriver.Chrome(service=path)

driver.get(main_url)
sleep(10)
driver.quit()

In [310]:
# test_links = []
path = Service(executable_path="D:\ChromeDriver\chromedriver.exe")
driver = webdriver.Chrome(service=path)

driver.get(main_url)

try:
    cookie = WebDriverWait(driver,7).until(
        EC.presence_of_element_located((By.ID, "onetrust-accept-btn-handler"))
    )
    cookie.click()
    
    content = WebDriverWait(driver,7).until(
        EC.presence_of_element_located((By.ID, "__next"))    
    )
    
    action_key_down_w = ActionChains(driver).key_down("w")
    action_key_up_w = ActionChains(driver).key_up("w")

    endtime = time.time() + 1.0

    while True:
        action_key_down_w.perform()
        
        if time.time() > endtime:
            action_key_up_w.perform()
            break
        
    sleep(240)
    
    soup = bs(driver.page_source, "html5lib")
    print(soup)
    
except:
    driver.quit()
    
    


#test_search_table = driver.find_element(By.CLASS_NAME,"css-14cy79a e3x1uf06")
#     for test_element in search_table:
#         offers = element.find_all(class_="css-p74l73 es62z2j17")
#         for offer in offers:
#             offer = offer.find("a", href = True, class_="css-rvjxyq es62z2j14")
            
#             offer_links.append(offer['href'])

# print(test_search_table)

In [219]:
#Returns the document with HTML code of requested webpage

def downloadPage(url):
    req = Request(url, headers={
        'Accept': "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8", 
        'Accept-Encoding': "gzip, deflate, br", 
        'Accept-Language': "pl,en-US;q=0.7,en;q=0.3", 
        'Host': "www.otodom.pl", 
        'Referer': "https://www.google.pl/", 
        'Sec-Fetch-Dest': "document", 
        'Sec-Fetch-Mode': "navigate", 
        'Sec-Fetch-Site': "cross-site", 
        'Sec-Fetch-User': "?1", 
        'Upgrade-Insecure-Requests': "1", 
        'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36", 
    })  
    webpage = urlopen(req).read() 
    doc = bs(webpage, 'html5lib')
    
    return doc

In [220]:
#Returns the list of offers from one page

def getOfferLink(main_url):
    offer_links = []
    htmlPage = downloadPage(main_url)
    
    search_table = htmlPage.find_all(class_="css-14cy79a e3x1uf06")
    for element in search_table:
        offers = element.find_all(class_="css-p74l73 es62z2j17")
        for offer in offers:
            offer = offer.find("a", href = True, class_="css-rvjxyq es62z2j14")
            
            offer_links.append(offer['href'])
            
    return offer_links

In [237]:
#Returns full list of searched offers 

def getListOfOffersLinks(pageNo_End, main_url):
    offer_links_list = []
    for pageNo in range(1, pageNo_End+1):
        try: 
            linkslist = getOfferLink(main_url)
            
            offer_links_list.extend(linkslist)
            time.sleep(randint(10,180))
            
        except:
            pass
        
    return offer_links_list

In [205]:
#Returns offer price

def getOfferPrice(doc):
    pice_location = doc.find("strong", class_="css-8qi9av eu6swcv19").get_text()
    offer_price = int(str(pice_location).split(" zł")[0].replace(" ",""))
    
    return offer_price

In [207]:
#Returns more offer details 

def getMoreDetails(doc):
    script = str(doc.find_all("script")[-1]).split(">")[1].split("<")[0]
    jsonfile = json.loads(script)
    details = jsonfile['props']['pageProps']['ad']['topInformation']

    return details

In [ ]:
#Te detale trzeba jeszcze zamienić w listę !!!!!!!!!

In [262]:
# Napraw to
check = pd.DataFrame(details).transpose()
print(check)

           0     1          2           3                         4  \
label   area  rent  rooms_num     deposit                     floor   
values  [40]    []        [2]  [3 200 zł]  [floor_no::floor_1, /10]   
unit      m²                                                          

                                5          6                        7  \
label               building_type  free_from                  outdoor   
values  [building_type-15::block]         []  [extras_types::balcony]   
unit                                                                    

                      8                                       9  
label   remote_services                     construction_status  
values               []  [construction_status-67::ready_to_use]  
unit                                                             


In [ ]:
#Do zrobienia jeszcze funkcja, która będzie zwracać DataFrame z detalami ofert z listy

In [258]:
#Pobieranie strony oferty - to check and fix

for link in offer_links:
    offer_url = f"https://www.otodom.pl/{link}"
    
    try:
        url_doc = downloadPage(offer_url)
        price = getOfferPrice(url_doc)
        details = getMoreDetails(url_doc)
        
        time.sleep(randint(10,200))
        
    except:
        pass

In [ ]:
# Co więcej trzeba zrobić: 
#     - dodać proxyRotation - chyba już wiem, jak to zrobić
#     - ogarnąć Selenium, żeby obejść Reacta